# Legacy MPRester Tutorial

In this notebook we will cover the basics of using the legacy MPRester API in order to load materials science data. MPRester has two APIs. One is the current modern version and the other is the legacy API. There are notebooks for both. Most of the notebooks in this course will be using the legacy API. 

#### Video

https://www.youtube.com/watch?v=Vuu7bNzmL8g&list=PLL0SWcFqypCl4lrzk1dMWwTUrzQZFt7y0&index=8 (Materials Data Access (Materials Project API Example)


Reminder, when running this example I need to open cmd line for miniconda since that's where I installed pymatgen module. From cmd line the following command "conda activate my_pymatgen" will open the environment followed by "Jupyter Notebook" to open the Jupyter platform.

In [ ]:
#Sparks_API = 'XXXXX' #put your API here

In [ ]:
import pandas as pd
from pymatgen.ext.matproj import MPRester
import os
# Copy and paste the path to your old API key file here. If you do not have an API key file, you can create a text file and paste the key into it.
filename = r'G:\My Drive\teaching\5540-6640 Materials Informatics\old_apikey.txt'

def get_file_contents(filename):
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


Sparks_API = get_file_contents(filename)

In [ ]:
import pymatgen.core as mg
si = mg.Element("Si")
print('Silicon has atomic mass of:', si.atomic_mass)

The API for Materials Project recently (2022) was updated. You can read about the differences between new and old with API Key and install instructions for each.
https://docs.materialsproject.org/downloading-data/differences-between-new-and-legacy-api

For this class, let's use the new API which you can read about here https://api.materialsproject.org/docs

To install, in miniconda in your My_Pymatgen environment run the command 'pip install mp-api'

First, I had to update pydantic with the following code 'pip install pydantic==2.0'



In [ ]:
#this works with the old legacy MPRester
from pymatgen.ext.matproj import MPRester

with MPRester(Sparks_API) as mpr:
    list_ids = mpr.get_materials_ids("TaC")
    print('All compounds in the Ta-C system are: ', mpr.get_materials_ids('Ta-C'))

Or we can pull data from a specific materials project id

In [ ]:
for i_d in list_ids:
    with MPRester(Sparks_API) as mpr:
        structure = mpr.get_structure_by_material_id(i_d)
        print(structure)


with MPRester(Sparks_API) as mpr:
    structure = mpr.get_structure_by_material_id('mp-1086')
    print(structure)
    cohesive_energy = mpr.get_cohesive_energy('mp-1086')
    print(cohesive_energy)


We can search for a materials project entry using a cif card

In [ ]:
import os
mpr = MPRester(Sparks_API)
ciffile = os.path.join('9008565.cif')
data1 = mpr.find_structure(ciffile)
print(data1)

How do we do queries though? What if we want to find all carbides having either Ta, Nb, or W?
We need to use the MPRester.query method!
This uses MongoDB's query syntax  --> mpr.query(criteria, properties)

In [ ]:
mpr = MPRester(Sparks_API)
#mpr.query('SiO2',['material_id','density'])
#mpr.query('**O3',['pretty_formula','material_id'])
mpr.query({'chemsys':'O-Si'},['material_id'])

In [ ]:
#example from John Dagdelen 2019 MP Workshop https://youtu.be/dsDOa_fgE7A
import matplotlib.pyplot as plt
%matplotlib inline
with MPRester(Sparks_API) as mpr:
    entries = mpr.query('Ta-C',['material_id','spacegroup.crystal_system'])
    systems = [entry['spacegroup.crystal_system'] for entry in entries]
    counts = {}
    
    for system in systems:
        if system in counts:
            counts[system] += 1
        else:
            counts[system] = 1    
    plt.bar(range(len(counts)), counts.values())
    plt.xticks(range(len(counts)), [key[0:7] for key in counts.keys()])
    

In [ ]:
#example from Wenhao Sun https://wenhaosun.github.io/docs/MSE593/
#from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
#import math
#This initializes the REST adaptor. Put your own API key in.
#mpr = MPRester(Sparks_API)
 
#Entries are the basic unit for thermodynamic and other analyses in pymatgen.
#This gets all entries belonging to the Ca-O system.
#entries = mpr.get_entries_in_chemsys(['Ca', 'La', 'S'])

#With entries, you can do many sophisticated analyses, 
#like creating phase diagrams.
#pd = PhaseDiagram(entries)
#plotter = PDPlotter(pd)
#plotter.show()

In [ ]:
#entries = mpr.get_entries_in_chemsys(['Mo', 'B', 'C'])

#With entries, you can do many sophisticated analyses, 
#like creating phase diagrams.
#pd = PhaseDiagram(entries)
#plotter = PDPlotter(pd)
#plotter.show()

In [ ]:
#example from Wenhao Sun https://wenhaosun.github.io/docs/MSE593/
from pymatgen.ext.matproj import MPRester

mpr = MPRester(Sparks_API)

criteria = {'elements':{"$in":["Li", "Na", "K"], "$all": ["O"]}, #All compounds contain O, and must have Li or Na or K
			'nelements':3,
            'icsd_ids': {'$gte': 0},
            'e_above_hull': {'$lte': 0.01},
            'anonymous_formula': {"A": 1, "B": 1, "C": 3},
            "band_gap": {"$gt": 1}
            }

        # The properties and the criteria use MaterialsProject features 
        # You can see what is queryable from the MP API documentation: 
        # https://github.com/materialsproject/mapidoc/tree/master/materials
        
        # The criteria uses mongodb query language. See here 
        # for more details: https://docs.mongodb.com/manual/reference/operator/query/

props = ['structure', "material_id",'pretty_formula','e_above_hull',"band_gap","band_structure"]
entries = mpr.query(criteria=criteria, properties=props)

print(len(entries))

for e in entries:
    print(e['pretty_formula'])
    print(e['band_gap'])
    print(e)
    break

# Now you try it!

Let's find your favorite structure in the Crystallographic Open Database, and then use MPRester API to find all entries in the Materials Project that have the same structure type


In [ ]:
#hint! the .get_structure_by_material_id() method is useful here
#hint! the .get_space_group_info() method is useful here
#hint! the .get_entry_by_material_id() method is useful here
#hint! consider using a search criteria that enforces the same space group and the same anonymous formula

